## Boosting

### What is boosting

Boosting is the idea of using multiple weak classifiers to generate strong classifiers.

The first round of training in the lift model uses different subsets of samples to train the base classifier. After training, the first round of the base classifier is used to classify and obtain the results of the first round of classification. Continue training after weighting samples that were misclassified in the first round of classification results. Repeat the n-wheel and the fusion model that the training will eventually get

### Adaboost

Three step in adaboost:

1. Initialize the weight distribution of the training data


2. Train with a training dataset with weight distribution to get the basic classifier.
   
  Calculate the classification error rate on the training dataset
  
  Update the weight distribution of the training dataset. 
  
  
3. Creates a linear combination of classifiers with weights.

### An adaboost implement by python:

ref:https://www.jianshu.com/p/ef16be5b66f4

In [1]:
def baseClassify(dataMartix, dim, threshVal, threshIneq):
    ret = ones((shape(dataMaritx)[0],1))
    if threshIneq=='lt':#less
        ret[dataMatrix[:,dim]<=threshVal]=-1.0
    else:
        ret[dataMatrix[:,dim]>threshVal]=-1.0
    return ret

In [2]:
def buildStump(dataArr,classLabels,D):
#将数据集和标签列表转为矩阵形式
    dataMatrix=mat(dataArr);labelMat=mat(classLabels).T
    m,n=shape(dataMatrix)
    numSteps=10.0;bestStump={};bestClasEst=mat(zeros((m,1))) 
    #最小错误率初始化为+∞
    minError=inf
    
    #特征->特征的步长->阈值对比方式
    for i in range(n):
        rangeMin=dataMatrix[:,i].min();rangeMax=dataMatrix[:,i].max()
        stepSize=(rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):
            for inequal in ['lt','gt']:
                threshVal=rangeMin+float(j)*stepSize
                predictedVals=\
                    stumpClassify(dataMatrix,i,threshVal,inequal)
                errArr=mat(ones((m,1)))
                errArr[predictedVals==labelMat]=0
                #计算"加权"的错误率
                #这里终于看懂了！！
                weigthedError=D.T*errArr
                if weigthedError<minError:
                    minError=weigthedError
                    bestClasEst=predictedVals.copy()
                    bestStump['dim']=i
                    bestStump['thresh']=threshVal
                    bestStump['ineq']=inequal
    #返回最佳单层决策树相关信息的字典，最小错误率，决策树预测输出结果
    return bestStump,minError,bestClasEst

In [ ]:
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    weakClassArr=[];m=shape(dataArr)[0]
    D=mat(ones((m,1))/m) #初始化样本权重
    aggClassEst=mat(zeros((m,1)))
    for i in range(numIt):
        bestStump,error,classEst=buildStump(dataArr,classLabels,D)
        alpha = float(0.5*log((1.0-error)/max(error,1e-16))) #max(error,1e-16)防止0误差的计算溢出
        bestStump["alpha"]=alpha
        weakClassArr.append(bestStump)
        #print ("D:",D.T,"\n","predClass:",classEst.T)
        
        #为下一次迭代更新D(很关键，矩阵运算容易写错！)
        expon=multiply(-1*alpha*mat(classLabels).T,classEst) #shape(5,1)*shape(5,1)对应元素相乘，得到各样本的-alpha*yi*G(xi)
        D=multiply(D,exp(expon)) #shape(5,1)*shape(5,1)对应元素相乘，
        D=D/D.sum() #得到各样本更新后的wi
        
        #计算汇集的分类结果aggClassEst(即，将每次最佳决策树分类结果bestClass*alpha相加)
        aggClassEst+=alpha*classEst
        #print("aggClassEst:",aggClassEst.T)
        aggErrors=multiply(sign(aggClassEst)!=mat(classLabels).T,ones((m,1)))
        errorRate=aggErrors.sum()/m
        #print("total error:",errorRate)
        if errorRate==0.0:
            break
    return weakClassArr,errorRate,aggClassEst